In [8]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (9.0s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [6]:
import optionspricing
import boto3
import helpers
import pandas
from dotenv import load_dotenv

load_dotenv()

BINANCE_SYMBOL = 'ETHUSDT'
quote_in_usd = False

s3 = boto3.resource('s3')

data_file = helpers.fetch_object(s3, "test-binance-prices-255120844515", f"{BINANCE_SYMBOL}-full.csv.zip")
prices_df = pandas.read_csv(data_file, compression='zip', header=0, index_col="dateTime")
prices_df.index = pandas.to_datetime(prices_df.index)

## Option valuation model: input parameters here

In [7]:
target_period_hours = 1 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2021, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size)
simulation = trading_model.simulate_strategy_long_straddle(option_chain_df, strikes_universe_size, quote_in_usd=quote_in_usd)
print(simulation)

target expiry: Mon 18 Mar, 08:00 (18 hours left)
current price: 3565.29
-------------------------------
hit ratio: 34%
cost: 0.020 / value: 0.020, gain% = -0.05%
($) cost: 73.09 / value: 71.29, average gain = -1.80
buy put 3550.0
buy call 3600.0
-------------------------------
hit ratio: 32%
cost: 0.015 / value: 0.015, gain% = 0.04%
($) cost: 53.48 / value: 54.88, average gain = 1.40
buy put 3525.0
buy call 3625.0
-------------------------------
hit ratio: 29%
cost: 0.011 / value: 0.012, gain% = 0.11%
($) cost: 38.51 / value: 42.41, average gain = 3.91
buy put 3500.0
buy call 3650.0
-------------------------------
hit ratio: 25%
cost: 0.008 / value: 0.009, gain% = 0.10%
($) cost: 29.24 / value: 32.87, average gain = 3.63
buy put 3475.0
buy call 3675.0


In [8]:
simulation.options_chain

,value_call,value_call_pct,call_bid,call_ask,value_put,value_put_pct,put_bid,put_ask
strike,,,,,,,,
3475.0,113.335095,NaN,0.0070,NaN,16.694296,0.004682,0.0041,0.0049
3500.0,93.136215,NaN,0.0215,0.0285,21.495416,0.006029,0.0050,0.0060
3525.0,74.519162,NaN,NaN,0.0300,27.878363,0.007819,0.0070,0.0080
3550.0,58.144609,NaN,0.0150,0.0170,36.503810,0.010239,0.0090,0.0110
3575.0,44.852502,0.012580,0.0110,0.0130,48.211703,NaN,0.0125,0.0140
3600.0,34.785057,0.009757,0.0085,0.0095,63.144258,NaN,0.0140,0.0190
3625.0,26.997842,0.007572,0.0060,0.0070,80.357043,NaN,0.0006,NaN
3650.0,20.914908,0.005866,0.0041,0.0048,99.274109,NaN,0.0220,0.0295
3675.0,16.170905,0.004536,0.0027,0.0033,119.530106,NaN,0.0290,0.0350


# Sanity checks

In [9]:
size = 8
option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size=size)
put_weights = [0.] * (2 * size + 1)
call_weights = [0.] * (2 * size + 1)
put_weights[size - 8] = 1.
put_weights[size - 2] = -1.
call_weights[size + 2] = -1.
call_weights[size + 8] = 1.

simulation = trading_model.simulate_strategy(option_chain_df, put_weights, call_weights, quote_in_usd=quote_in_usd)
print(simulation)

target expiry: Mon 18 Mar, 08:00 (18 hours left)
current price: 3565.29
-------------------------------
hit ratio: 61%
cost: -0.010 / value: -0.012, gain% = -0.23%
($) cost: -36.01 / value: -44.20, average gain = -8.19
buy put 3350.0
sell put 3525.0
sell call 3625.0
buy call 3775.0
